In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance
import colorsys
import skimage.io
from os import listdir
from os.path import isfile, join
import os
import os.path

In [190]:
extractedPath = 'results/KI67/extractedSignal/P10/aorta4/ki67Signal_P10_aorta4_NEW2.csv'
df = pd.read_csv(extractedPath)
dfThresholded = pd.DataFrame()

In [ ]:
brighteningValue = 700
imageNumber = 15

segFolder = 'segmentations/ki67_tiled/P14/aorta4'
seg1 = 'ki67Image_P14_aorta4_nuclei_'  
seg2 = '_seg.npy'
ki671 = 'preprocessedImage/ki67/tiled/ki67/P14/aorta4/ki67Image_P14_aorta4_ki67_' 
ki672 = '.tif'

segFilePath = segFolder + '/' + seg1 + str(imageNumber)  + seg2
ki67ImagePath = ki671 + str(imageNumber) + ki672

nucMasks = np.load(segFilePath, allow_pickle=True).item()['masks']

ki67Image = skimage.io.imread(ki67ImagePath)
dfImage = df.loc[df.imageNumber == imageNumber]


In [ ]:
thresholdIntensity = 0.1
thresholdfraction = 3

dfFilter = dfImage.loc[(dfImage.fractionNorm >= thresholdfraction) & (dfImage.medianNorm >= thresholdIntensity)]
dfUnfiltered = dfImage.loc[~((dfImage.fractionNorm >= thresholdfraction) & (dfImage.medianNorm >= thresholdIntensity))]


backgroundMaskRedInt = np.where(nucMasks > 0, np.median(ki67Image), ki67Image)
ki67Bright = np.where((ki67Image.max() - ki67Image) < brighteningValue,ki67Image.max(),ki67Image+brighteningValue)

activeImage = np.zeros_like(ki67Image)
inactiveImage = np.zeros_like(ki67Image)
activeCoord = np.isin(nucMasks, dfFilter['label'].values)
inactiveCoord = ~np.isin(nucMasks, dfFilter['label'].values)
activeImage[activeCoord] = ki67Bright[activeCoord]
inactiveImage[inactiveCoord] = ki67Bright[inactiveCoord]

plt.figure(figsize=(16, 8))

#plot original image
plt.imshow(ki67Bright , cmap="gray")
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
#plot extraced nuclei and image withour extracted nuclei
axes[0].imshow(activeImage, cmap='gray')
axes[1].imshow(inactiveImage, cmap='gray')

plt.show()

In [ ]:
plt.hist(df.medianNorm, bins = 40, range = (0,0.1))

In [ ]:
thresholdIntensity = 0.02
thresholdfraction = 3
df['thresholdfraction'] = thresholdfraction
df['thresholdIntensity'] = thresholdIntensity
df['positive'] = ((df['fractionNorm'] > thresholdfraction) & (df['medianNorm'] > thresholdIntensity)).astype(int)
np.mean(df['positive'])

dfSave = df

In [192]:
dfSave.to_csv('/results/KI67/thresholdedValues/P10/aorta04/ki67Signal_P14_aorta04.csv', index=False)

In [ ]:
filesNuclei = [f for f in listdir(segFolder) if isfile(join(segFolder, f)) if "_seg.npy" in f if "._" not in f]
imageNumbers = []
for file in filesNuclei:
    imageNumbers.append(int(file.split(seg2)[0].split(seg1)[-1]))

df['thresholdfraction'] = thresholdfraction
df['thresholdIntensity'] = thresholdIntensity
df['positive'] = ((df['fractionNorm'] > thresholdfraction) & (df['medianNorm'] > thresholdIntensity)).astype(int)
dfSave = df[df['imageNumber'].isin(imageNumbers)]


In [ ]:
dfSave.to_csv('/results/KI67/thresholdedValues/P21/aorta04/ki67Thresholded_P14_a4.csv', index=False)